## Fitting profiles with Markov Chain Monte Carlo
This is a jupyter notebook intended to fit stellar density profiles to 
a bunch of clusters simulated. It fits the exponential radius, ellipticity

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import emcee
import astropy.io.fits as fits
import corner

ModuleNotFoundError: No module named 'emcee'

Below are a few functions (that should be documented) to provide the results of the fitting.

In [ ]:
def lnprior(theta):
    re, ell, th, bg, xc, yc = theta
    if 0.01 < re < 20. and 0.0 < ell < 0.3 and -1.8 < th < 1.8 and 0. < bg < 500. and -3. < xc < 3. and -3. < yc < 3.:
        return 0
    return -np.inf


def lnlike(theta, RA, DEC):
    re, ell, th, bg, xc, yc = theta
    dX = (RA-(ra0+xc/60.))*np.cos(np.radians(dec0))
    dY = (DEC-(dec0+yc/60.))
    ri = 60.*np.sqrt(((dX*np.cos(np.radians(100*th))-dY*np.sin(np.radians(100*th)))/(1.-ell))
                     ** 2.+(dX*np.sin(np.radians(100*th))+dY*np.cos(np.radians(100*th)))**2.)
    s0 = (len(RA)-bg*3600.)/((1-ell)*2.*np.pi*re**2)
    if not np.isfinite(2.*np.sum(np.log(bg+s0*np.exp(-ri/re)))):
        return -np.inf
    return 2.*np.sum(np.log(bg+s0*np.exp(-ri/re)))


def lnprob(theta, RA, DEC):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(theta, RA, DEC)


# Some definitions:
R0 = 1.0
# ra0, dec0, reinf, resup, thmin, thmax = 67.3156, -23.8368, 0.1, 10., 0.0, 180.
HPX_, ra0_, dec0_, re_kick_, ell_kick_, pa_kick_, dist = np.loadtxt(
    'star_clusters_simulated.dat', usecols=(0, 6, 7, 8, 9, 10, 12), unpack=True)

# Converting exponential radius from parsecs to arcmin
re_kick = 60. * (180. / np.pi) * np.arctan(re_kick_ / dist)

hdu = fits.open('des_mockcat_for_detection.fits', memmap=True)
RA = hdu[1].data.field('ra')
DEC = hdu[1].data.field('dec')
MAGG = hdu[1].data.field('mag_g_with_err')
MAGGERR = hdu[1].data.field('magerr_g')
MAGR = hdu[1].data.field('mag_g_with_err')
MAGRERR = hdu[1].data.field('magerr_r')
hdu.close()

print('                 HPX      re          ell           pa_kick_     robg   shift_ra    shift dec')
for i in range(len(ra0_)):
    print('Pars (ideal):', HPX_[i], re_kick[i], ell_kick_[i], pa_kick_[i], robg, '0.00', '0.00')
    reinf = 0.5 * re_kick[i]
    resup = 1.5 * re_kick[i]
    thmin = pa_kick_[i] - 45.
    thmax = pa_kick_[i] + 45.
    ra0 = ra0_[i]
    dec0 = dec0_[i]

    R = 60.*np.sqrt(((np.cos(np.radians(dec0))*(ra0-RA))**2.)+(DEC-dec0)**2)

    re_ml = (reinf + resup)/2.
    ell_ml = 0.1
    th_ml = (thmin + thmax)/200.
    idbg = (R > R0)
    robg = float(len(RA[idbg]))/(3600.-np.pi*(R0)**2)  # stars/arcmin^2
    N_star = (len(RA)-robg*3600.)
    s_ml = N_star/(2.*np.pi*re_ml**2)
    xc_ml = 0.0
    yc_ml = 0.0
    result = re_ml, ell_ml, th_ml, robg, xc_ml, yc_ml
    print(HPX_[i], re_ml, ell_ml, th_ml, robg, xc_ml, yc_ml)

    ndim, nwalkers = 6, 300  # 300
    pos = [result + 1e-4*np.random.randn(ndim) for j in range(nwalkers)]
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(RA, DEC))
    sampler.run_mcmc(pos, 500)  # 500
    samples = sampler.chain[:, 100:, :].reshape((-1, ndim))

    samples[:, 2] = 100.*(samples[:, 2])
    re_mcmc, ell_mcmc, th_mcmc, robg_mcmc, xc_mcmc, yc_mcmc = map(lambda v: (
        v[1], v[2]-v[1], v[1]-v[0]), zip(*np.percentile(samples, [16, 50, 84], axis=0)))

    print(HPX_[i], re_mcmc[0], ell_mcmc[0], th_mcmc[0], robg_mcmc[0], xc_mcmc[0], yc_mcmc[0])
# Plotting data
    fig = corner.corner(samples, labels=["$re$", "$ell$", "$th$", r"$\rho_{bg}$", r"$\Delta\alpha$", r"$\Delta\delta$"], truths=[
                        re_mcmc[0], ell_mcmc[0], th_mcmc[0], robg_mcmc[0], xc_mcmc[0], yc_mcmc[0]], quantiles=[0.16, 0.5, 0.84], show_titles=True, plot_contours=True)
    plt.savefig(str(HPX_[i]) + '_plus.png')
    plt.close()

    L = np.zeros(len(RA))
    dX = (RA-(ra0+xc_mcmc[0]/60.))*np.cos(np.radians(dec0))
    dY = (DEC-(dec0+yc_mcmc[0]/60.))
    ri = 60.*np.sqrt(((dX*np.cos(np.radians(100*th_mcmc[0]))-dY*np.sin(np.radians(100*th_mcmc[0])))/(
        1.-ell_mcmc[0]))**2.+(dX*np.sin(np.radians(100*th_mcmc[0]))+dY*np.cos(np.radians(100*th_mcmc[0])))**2.)
    L = (np.exp(-ri/re_mcmc[0]))

#a = open('J0429_stars_list_profile.dat','w')
# for j in range(len(RA)):
#    if (L[j]<0.):
#        L[j]=0.
#    print >> a, RA[j], DEC[j], MAGG[j], MAGR[j], ERRG[j], ERRR[j], SM[j], L[j]
# a.close

35934.0 0.15359148315490373 0.09 105.53 robg 52.7344 -28.631
0.15359148315490373 0.1 1.0553 100.49797880886848 0.0 0.0


/tmp/ipykernel_23245/870490835.py:15: RuntimeWarning: invalid value encountered in log
  if not np.isfinite(2.*np.sum(np.log(bg+s0*np.exp(-ri/re)))):


35934.0 0.05196043027156208 0.04323218081775747 130.3384156366609 100.4415760110896 -0.0003095377948625984 0.0021161296746692758
35935.0 0.22917372628576235 0.02 78.61 robg 53.4375 -27.9532
0.22917372628576235 0.1 0.7861 100.49047225816723 0.0 0.0


/tmp/ipykernel_23245/870490835.py:15: RuntimeWarning: invalid value encountered in log
  if not np.isfinite(2.*np.sum(np.log(bg+s0*np.exp(-ri/re)))):
